In [1]:
!pip install fastai==2.7.9
!pip install fastkaggle==0.0.7
!pip install fasterai==0.1.11
!pip install ipython-autotime==0.3.1
!pip install torch==1.12.1
%load_ext autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.5/225.5 kB 11.6 MB/s eta 0:00:00
  Attempting uninstall: fastai
    Found existing installation: fastai 2.7.12
    Uninstalling fastai-2.7.12:
      Successfully uninstalled fastai-2.7.12
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 1.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: torch
    Found existing installation: torch 1.13.0
    Uninstalling torch-1.13.0:
      Successfully uninstalled torch-1.13.0
time: 497 µs (started: 2023-04-28 09:39:44 +00:00)


In [2]:
from fastai.vision.all import *
from fastai.callback.all import *
from fastai.data.all import *
from fastai.imports import *
from fastai.torch_core import *
from fastai.learner import *
from fastai.fp16_utils import *
from fastai.callback.fp16 import *
import fastai.optimizer as optim


from fasterai.sparse.all import *
from fasterai.core.criteria import *
from fasterai.regularize.all import *

from fastkaggle import *

import torch
import torch.nn as nn
import torch.nn.functional as F

set_seed(42)

/opt/conda/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /opt/conda/lib/python3.7/site-packages/torchvision/image.so: undefined symbol: _ZN3c104impl8GPUTrace13gpuTraceStateE
  warn(f"Failed to load image Python extension: {e}")


time: 2.95 s (started: 2023-04-28 09:39:44 +00:00)


In [3]:
comp = 'paddy-diseases/paddy-diseases'

path = setup_comp(comp, install='"fastcore>=1.4.5" "fastai>=2.7.1" "timm>=0.6.2.dev0"')

time: 10.6 s (started: 2023-04-28 09:39:47 +00:00)


In [4]:
dls = ImageDataLoaders.from_folder(path/"train", 
                                   valid_pct=0.25, 
                                   item_tfms=Resize(256), 
                                   batch_tfms=
                                   aug_transforms(   size=64, 
                                                                min_scale=0.75,
                                                                max_rotate=10., 
                                                                min_zoom=0.8, 
                                                                max_zoom=1.2, 
                                                            
                                                            ),
                                   bs=32)

time: 7.41 s (started: 2023-04-28 09:39:57 +00:00)


In [5]:
model = vgg16_bn().cuda()

time: 2.11 s (started: 2023-04-28 09:40:05 +00:00)


In [6]:
cbs = [     
#         GradientAccumulation(32), 
#         SparsifyCallback(sparsity=12.5, 
#                          granularity='filter', 
#                          context='local', 
#                          criteria=large_final, 
#                          schedule=one_cycle),
        ]

time: 684 µs (started: 2023-04-28 09:40:07 +00:00)


In [7]:
from fastai.learner import Learner

class CustomLearner(Learner):
    def _step(self):
        before_memory = torch.cuda.max_memory_allocated(device=torch.device("cuda"))
        super()._step()
        after_memory = torch.cuda.max_memory_allocated(device=torch.device("cuda"))
      
        # calculate the memory usage during the optimizer step
        optimizer_memory_usage = after_memory - before_memory
    
        print(f"Optimizer memory usage: {optimizer_memory_usage} Byte")


time: 3.62 ms (started: 2023-04-28 09:40:07 +00:00)


In [8]:
learn = CustomLearner(dls, 
                model = model, 
                metrics=[accuracy,
                         error_rate,
                        ],
               ).to_non_native_fp16()
learn.unfreeze()
learn.recorder.train_metrics = True
learn.path = Path("/kaggle/working/")



time: 10.5 ms (started: 2023-04-28 09:40:07 +00:00)


In [9]:
learn.fit_one_cycle(1, 3e-5, cbs=cbs)

epoch,train_loss,train_accuracy,train_error_rate,valid_loss,valid_accuracy,valid_error_rate,time
0,2.425689,0.183654,0.816346,1.976087,0.344231,0.655769,01:29


Optimizer memory usage: 1764478976 Byte
Optimizer memory usage: 169157120 Byte
Optimizer memory usage: 0 Byte
Optimizer memory usage: 0 Byte
Optimizer memory usage: 0 Byte
Optimizer memory usage: 0 Byte
Optimizer memory usage: 0 Byte
Optimizer memory usage: 0 Byte
Optimizer memory usage: 393216 Byte
Optimizer memory usage: 0 Byte
Optimizer memory usage: 0 Byte
Optimizer memory usage: 0 Byte
Optimizer memory usage: 0 Byte
Optimizer memory usage: 0 Byte
Optimizer memory usage: 0 Byte
Optimizer memory usage: 0 Byte
Optimizer memory usage: 0 Byte
Optimizer memory usage: 0 Byte
Optimizer memory usage: 0 Byte
Optimizer memory usage: 0 Byte
Optimizer memory usage: 0 Byte
Optimizer memory usage: 0 Byte
Optimizer memory usage: 0 Byte
Optimizer memory usage: 0 Byte
Optimizer memory usage: 0 Byte
Optimizer memory usage: 0 Byte
Optimizer memory usage: 0 Byte
Optimizer memory usage: 0 Byte
Optimizer memory usage: 0 Byte
Optimizer memory usage: 0 Byte
Optimizer memory usage: 0 Byte
Optimizer memory 